<a href="https://colab.research.google.com/github/fathanzys/NLP-Fathan/blob/NLP_Code/tugas2_0102522024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing

## Tugas 1: Regular Expressions & Model Bahasa N-gram

### Mekanisme

Anda hanya diwajibkan untuk mengumpulkan file ini saja ke uploader yang disediakan di https://elearning.uai.ac.id/. Ganti nama file ini saat pengumpulan menjadi **tugas1_NIM.ipynb**.

**Keterlambatan**: Pengumpulan tugas yang melebihi tenggat yang telah ditentukan tidak akan diterima. Keterlambatan akan berakibat pada nilai nol untuk tugas ini.

**Kolaborasi**: Anda diperbolehkan untuk berdiskusi dengan teman Anda, tetapi dilarang keras menyalin kode maupun tulisan dari teman Anda.

### Petunjuk

Pastikan jawaban Anda singkat, padat, dan jelas. Mayoritas pertanyaan yang diberikan dapat dijawab dalam 3-4 kalimat saja.

In [ ]:
import numpy as np
import pandas as pd
import re

## 1. Regular Expressions (5 poin)

### Soal 1.1 (2 poin)

Cari pola regular expression yang dapat mencocokkan semua elemen di dalam `words_a`, tetapi tidak cocok dengan apa pun di dalam `words_b`.

In [ ]:
words_a = [
    "membacakan",
    "menuliskan",
    "melihatkan",
    "mendengarkan" ,
    "mengajarkan",
    "menarikan",
    "menyanyikan" ,
    "memasakkan",
    "mencucikan",
    "membelikan",
    "menjualkan",
    "memberikan" ,
]

words_b = [
    "rumah",
    "meja",
    "kursi",
    "ikan",
    "semen",
    "ramen" ,
    "akan",
    "bukan" ,
    "mental",
]
pattern = r"me[a-z]+kan$" #<< jawabannya, karena
#pada words_a, semua kata berawalan "me", diikuti oleh hruf kecil (a-z) 1 atau lebih, dan diakhiri "kan", jadi cocok dengan regex.
#pada words_b, tidak ada pola seperti itu (tidak di mulai "me" lalu banyak huruf lalu "kan" di akhir), jadi tidak cocok dengan regex.

assert pd.Series(words_a).str.contains(pattern).all()
assert not pd.Series(words_b).str.match(pattern).any()

### Soal 1.2 (3 poin)

Cari pola regular expression yang dapat mencocokkan semua elemen di dalam `buah_a`, tetapi tidak cocok dengan apa pun di dalam `buah_b`. Anda hanya boleh menggunakan maksimal 6 karakter untuk pola yang dihasilkan.

In [ ]:
buah_a = [
    "pisang",
    "mangga",
    "rambutan",
    "durian",
    "salak",
    "duku",
    "kelapa",
    "sirsak",
    "belimbing",
    "manggis",
]

buah_b = [
    "pir",
    "stroberi",
    "ceri",
    "kiwi",
    "leci",
    "persik",
]

pattern = r"[aum]" #<< jawabannya, karena
#pada buah_a, karena ada huruf a/u/m dan cocok regex nya
#pada buah_b,  tidak cocok regex nya karena ngga ada huruf a/u/m

assert len(pattern) <= 6
assert pd.Series(buah_a).str.contains(pattern).all()
assert not pd.Series(buah_b).str.contains(pattern).any()

## 2. Model Bahasa N-gram (15 poin)

### Deskripsi Dataset

Dataset yang Anda akan gunakan dalam tugas ini adalah [Indonesian News Corpora 300K](https://wortschatz.uni-leipzig.de/en/download/Indonesian#ind_news_2024) tahun 2024 dari Leipzig Corpora Collection.

In [ ]:
sentences = pd.read_csv(
    "https://raw.githubusercontent.com/aliakbars/uai-nlp/refs/heads/main/datasets/ind_news_2024_300K-sentences.txt",
    sep="\\t",
    names=["index", "text"],
)
del sentences["index"]

<ipython-input-83-a2a6c86ed52e>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  sentences = pd.read_csv(


### Soal 2.1 (2 poin)

Bagilah dataset menjadi data latih dan data uji dengan rasio 80:20.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(sentences, test_size=0.2, random_state=42)
#melatih dan meng tes data
assert len(train) == 240000
assert len(test) == 60000

### Soal 2.2 (2 poin)

Buatlah tabel `unigram_df` yang menghasilkan **unigram** dari data latih yang telah dihasilkan di atas. Gunakan [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) untuk proses tokenisasi dan menghasilkan unigramnya, serta pola token default `'(?u)\\b\\w\\w+\\b'`.

Contoh hasil tabel:
| word      |   freq |
|:----------|-------:|
| diketahui |   2654 |
| layanan   |   2015 |
| maju      |   1386 |
| aksi      |   1707 |
| januari   |   1316 |

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w\w+\b")

#memlakukan transformasi pada data latih untuk menghasilkan representasi unigram
X_train = vectorizer.fit_transform(train['text'])

#menghitung jumlah kemunculan setiap kata (unigram)
word_counts = X_train.sum(axis=0)
words = vectorizer.get_feature_names_out()
freq = np.asarray(word_counts).flatten()

#membuat dataframe untuk menyimpan kata-kata dan frekuensinya
unigram_df = pd.DataFrame({"word": words, "freq": freq})
unigram_df = unigram_df.sort_values(by="freq", ascending=False).reset_index(drop=True)

#memastikan bahwa dataframe memiliki kolom, 'word' dan 'freq'
assert ["word", "freq"] == unigram_df.columns.to_list()

#menampilkan 5 baris pertama dari unigram_df
print(unigram_df.head())

    word   freq
0   yang  93794
1    dan  82683
2     di  80752
3    ini  42942
4  untuk  40433


### Soal 2.3 (3 poin)

Buatlah tabel `bigram_df` yang menghasilkan **bigram** dari data latih yang telah dihasilkan di atas. Gunakan [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) untuk proses tokenisasi dan menghasilkan bigram, serta pola token default `'(?u)\\b\\w\\w+\\b'`. Tambahkan `ooo` sebagai _start token_ dan `zzz` sebagai _end token_.

Contoh hasil tabel:
| word          | next_word   |   freq |
|:--------------|:------------|-------:|
| akan          | ada         |    461 |
| sebagai       | bentuk      |    365 |
| dua           | orang       |    363 |
| internasional | zzz         |    385 |
| kasus         | ini         |    535 |
| kami          | berharap    |    379 |
| informasi     | yang        |    398 |
| yang          | kita        |    465 |
| pada          | musim       |    325 |
| tahun         | zzz         |   1060 |

In [ ]:
def add_start_end_tokens(text):
    return "ooo " + text + " zzz"

train['text_with_tokens'] = train['text'].apply(add_start_end_tokens)

#menggunakan countvectorizer untuk ekstraksi bigram dari teks
vectorizer = CountVectorizer(
    token_pattern=r"(?u)\b\w\w+\b",  #pola regex
    ngram_range=(2, 2),
    max_features=10000,
    min_df=5
)

#transformasi pada datatrain untuk menghasilkan representasi bigram
X_train = vectorizer.fit_transform(train['text_with_tokens'])

#membuat dataframe yg menyimpang variabel biagram
bigram_df = pd.DataFrame(
    {
        'bigram': vectorizer.get_feature_names_out(),
        'freq': X_train.sum(axis=0).A1
    }
).sort_values(by="freq", ascending=False).reset_index(drop=True)  #mengurutkan berdasarkan freq (frekuensi)

#mempisahkan bigram menjadi dua kolom 'word' dan 'next_word'
bigram_df[['word', 'next_word']] = bigram_df['bigram'].str.split(expand=True)
bigram_df = bigram_df[['word', 'next_word', 'freq']]
bigram_df = bigram_df.dropna()

#memastikan bahwa kolom yang diinginkan ada dan sesuai
assert ["word", "next_word", "freq"] == bigram_df.columns.to_list()
assert "ooo" in bigram_df["word"].to_list()
assert "zzz" in bigram_df["next_word"].to_list()

#menampilkan 5 baris pertama dari bigram_df
print(bigram_df.head())

       word next_word   freq
0      2024       zzz  11777
1       ooo     namun   4605
2       ini       zzz   4525
3      saat       ini   4259
4  tersebut       zzz   4076


### Soal 2.4 (5 poin)

Implementasikan kode yang menghasilkan array berisi skor (*log probability*) jika diberikan sebuah kalimat. Petunjuk implementasi:
1. Gunakan tabel unigram dan bigram yang telah dihasilkan di soal sebelumnya.
1. Gunakan metode Stupid backoff yang menggunakan parameter `alpha` untuk mengatasi kasus bigram yang tidak ditemukan.
1. Gunakan metode Laplace smoothing untuk menangani kasus unigram yang tidak ditemukan.

In [ ]:
class BigramModel:
    def __init__(self, unigram_df: pd.DataFrame, bigram_df: pd.DataFrame, alpha: float = 0.4):
        #menginisialisasi model dengan data unigram dan bigram serta alpha untuk backoff
        self.unigram_df = unigram_df
        self.bigram_df = bigram_df
        self.alpha = alpha
        self.N = self.unigram_df.freq.sum()  #total dari frekuensi kata
        self.V = len(self.unigram_df)  #jumlah kata unik

    def score_unigram(self, word: str) -> float:
        #menghitung skor unigram menggunakan teknik laplace smoothing
        count = self.unigram_df.loc[self.unigram_df['word'] == word, 'freq'].sum()
        prob = (count + 1) / (self.N + self.V)
        return np.log(prob)

    def score(self, word: str, prev_word: str) -> float:
        #menghitung skor bigram atau menggunakan teknik backoff
        bigram_count = self.bigram_df[
            (self.bigram_df['word'] == prev_word) &
            (self.bigram_df['next_word'] == word)
        ]['freq'].sum()

        prev_count = self.unigram_df.loc[self.unigram_df['word'] == prev_word, 'freq'].sum()

        if bigram_count > 0 and prev_count > 0:
            prob = bigram_count / prev_count
            return np.log(prob)
        else:
            #teknik backoff menggunakan unigram dengan smoothing
            backoff_prob = self.alpha * ((self.unigram_df.loc[
                self.unigram_df['word'] == word, 'freq'
            ].sum() + 1) / (self.N + self.V))
            return np.log(backoff_prob)

    def sentence_score(self, sentence: str, n: int = 2) -> np.ndarray:
        #menghitung skor untuk seluruh kalimat berdasarkan bigram atau unigram
        words = sentence.strip().split()
        if n == 2:
            return np.array([self.score(words[1], words[i-1]) for i in range(1, len(words))])
        elif n == 1:
            return np.array([self.score_unigram(word) for word in words])
        else:
            raise NotImplemented("Saat ini hanya bisa untuk n=1 atau n=2")



bm = BigramModel(unigram_df, bigram_df, alpha=0.4)
assert bm.score_unigram("qqq") > bm.score("qqq", "xxx")
assert bm.score("qqq", "xxx") == np.log(
    0.4 * 1 / (unigram_df.freq.sum() + len(unigram_df))
)
assert bm.score("mana", "di") <= 0
assert bm.score("mungkin", "tidak") > bm.score("qqq", "xxx")
assert bm.score("satu", "salah") > bm.score("tangkap", "salah")
assert (bm.sentence_score("salah satu") <= 0).all()

print(bm.score_unigram("qqq"))
print(bm.score("qqq", "xxx"))

-15.200211900904204
-16.11650263277836


### Soal 2.5.a (2 poin)

Implementasikan fungsi untuk menghitung perplexity dalam logaritma. Fungsi ini hanya menerima masukan berupa array berisi $\log P(w_i)$.

$$
PP(W) = \sqrt[N]{\frac{1}{\prod_{i=1}^N P(w_i)}}
$$

In [ ]:
def perplexity(log_probs: np.ndarray):
    N = len(log_probs)  # menghitung jumlah elemen dalam log probabilitis
    log_sum = np.sum(log_probs)  # menjumlahkan semua nilai log probabilities
    return np.exp(-log_sum / N)  # menghiitung dan kembalikan perplexity dengan rumus yang sesuai

# memverifikasi dengan log probabilities yang semuanya 0, seharusnya hasilnya 1
assert perplexity(np.array([0., 0., 0.])) == 1
# memverifikasi dengan log probabilities -10 dan -20, seharusnya hasilnya exp(15)
assert perplexity(np.array([-10., -20.])) == np.exp(15)

### Soal 2.5.b (1 poin)

Berdasarkan kalimat contoh yang diberikan, periksa dan pastikan bahwa perplexity dari `BigramModel` lebih kecil daripada model unigram.

In [ ]:
example = "pengakuan tersebut diungkapkan dalam wawancara dengan media"

#score kalimat menggunakan Unigram
unigram_log_prob = []
for word in example.split():
    unigram_log_prob.append(bm.score_unigram(word))
unigram_log_prob = np.array(unigram_log_prob)

#score kalimat menggunakan bigram
bigram_log_prob = bm.sentence_score(example)

#menghitung perplexity
unigram_perplexity = perplexity(unigram_log_prob)
bigram_perplexity = perplexity(bigram_log_prob)

print("Perplexity Unigram:", unigram_perplexity)
print("Perplexity Bigram:", bigram_perplexity)

assert bigram_perplexity < unigram_perplexity

Perplexity Unigram: 1642.5858621370896
Perplexity Bigram: 579.6947395924054


### Soal 2.6 (3 poin)

Ambil 30 sampel dari data uji. Hitunglah nilai rata-rata berbobot (_weighted average_) perplexity per kalimat pada data uji dengan menggunakan jumlah kata per kalimat sebagai bobotnya.

Pastikan bahwa:
1. Metode tokenisasi dan preprocessing yang dilakukan sama dengan soal 2.2 dan 2.3.
1. Nilai perplexity dihitung untuk model unigram dan bigram dengan backoff.

_Petunjuk: Gunakan metode `.findall()`_

In [ ]:
def compute_perplexity(log_probs: np.ndarray, n_tokens: int) -> float:
    return np.exp(-log_probs.sum() / n_tokens)

# mengambil data dari sentences uji
sample_sentences = np.array(test).flatten()

# memiilih 30 kalimat acak dari data uji
np.random.seed(42)
selected_sentences = np.random.choice(sample_sentences, size=30, replace=False)

# variabel untuk menyimpan perplexity dan jumlah kata per kalimat
unigram_perplexities = []
bigram_perplexities = []
word_counts = []

# polaa regex untuk tokenisasi (ambil kata yang panjangnya >= 2 huruf)
token_pattern = r'\b\w\w+\b'

# memproses setiap kalimat
for sentence in selected_sentences:
    tokens = re.findall(token_pattern, sentence.lower())  # Tokenisasi kalimat
    if not tokens:
        continue

    num_tokens = len(tokens)

    # menghiitung log-probabilitas untuk unigram dan bigram
    unigram_log_probs = bm.sentence_score(sentence, n=1)
    bigram_log_probs = bm.sentence_score(sentence, n=2)

    # menghiititung perplexity untuk unigram dan bigram
    unigram_perplexity = compute_perplexity(unigram_log_probs, num_tokens)
    bigram_perplexity = compute_perplexity(bigram_log_probs, num_tokens)

    # menyimpan hasil perplexity dan jumlah kata
    unigram_perplexities.append(unigram_perplexity)
    bigram_perplexities.append(bigram_perplexity)
    word_counts.append(num_tokens)

# menghiitung rata-rata berbobot perplexity
total_words = np.sum(word_counts)
weighted_unigram_avg = np.sum(np.array(unigram_perplexities) * np.array(word_counts)) / total_words
weighted_bigram_avg = np.sum(np.array(bigram_perplexities) * np.array(word_counts)) / total_words

# menampilkan hasil rata-rata perplexity
print("rata-rata berat perplexity unigram :", weighted_unigram_avg)
print("rata-rata berat perplexity bigram :", weighted_bigram_avg)

rata-rata berat perplexity unigram : 211752.0274204288
rata-rata berat perplexity bigram : 4688909.321733292


### Soal 2.7 (2 poin)

Berikan kesimpulan dari analisis yang sudah dilakukan pada bagian kedua dari tugas ini.

Saya mempelajari dan mendapatkan insight dari tugas ini dimulai dengan mencari regular expression (regex) dan menemukan pola dari data yang sudah diberikan, seperti pada soal 1.1 dengan mencari pola yang cocok untuk words_a tetapi tidak cocok dengan words_b, dan pada soal 1.2 dengan mencari pola yang cocok untuk buah_a tetapi tidak cocok dengan buah_b, dengan batasan 6 karakter (<=). Pola regex yang ditemukan adalah r"[aum]", yang mencocokkan kata yang mengandung huruf 'a', 'u', atau 'm'.

Selanjutnya, pada soal 2 yang membahas tentang model N-gram, saya memulai dengan membagi dataset menjadi data latih (240.000 baris) dan data uji (60.000 baris) pada soal 2.1. Pada soal 2.2, saya menghitung frekuensi kata yang muncul pada teks menggunakan CountVectorizer, yang mengubah teks menjadi angka untuk menunjukkan seberapa sering kata-kata muncul dalam dataset. Pada soal 2.3, saya menambahkan token 'ooo' dan 'zzz' sebagai penanda awal dan akhir kalimat, kemudian membuat kolom 'word', 'next_word', dan 'freq' untuk mencatat frekuensi kemunculan kata-kata dan pasangan kata.

Pada soal 2.4, saya mempelajari implementasi model bigram dengan menggunakan teknik Laplace Smoothing dan Stupid Backoff untuk menghitung skor probabilitas unigram dan bigram. Hasil skor yang diperoleh (-15.200211900904204 dan -16.11650263277836) menunjukkan nilai negatif, yang mengindikasikan bahwa probabilitas urutan kata yang dihitung oleh model relatif rendah. Di soal 2.5a, saya mempelajari fungsi perplexity untuk menghitung tingkat kompleksitas model berdasarkan log probabilitas. Perplexity dihitung dengan dua contoh: pertama, dengan log probabilitas 0 (hasilnya 1 karena log(1) = 0), dan kedua, dengan log probabilitas -10 dan -20 (hasilnya exp(15)). Nilai perplexity yang lebih rendah menunjukkan model yang lebih baik dalam memprediksi urutan kata, sementara nilai yang lebih tinggi menunjukkan model yang kurang efektif.

Pada soal 2.5b, saya menghitung perplexity untuk model unigram dan bigram. Proses ini melibatkan perhitungan probabilitas unigram dan bigram pada setiap kata dalam kalimat, kemudian menghitung log probabilitas dan perplexity untuk masing-masing model. Pada soal 2.6, saya menghitung rata-rata perplexity untuk model unigram dan bigram dengan menggunakan 30 kalimat acak dari data uji. Kalimat-kalimat ini diproses melalui tokenisasi dengan regex untuk kata yang memiliki panjang minimal 2 huruf, dan log-probabilitas dihitung untuk masing-masing model. Hasil rata-rata perplexity yang diperoleh adalah: untuk unigram 211752.0274204288 dan untuk bigram 4688909.321733292.